In [3]:
# -*- coding:utf-8 -*-
import json
import requests
import csv
import re
from lxml import etree
from datetime import date,datetime
import pandas as pd
import math
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
interest = html.xpath('.//td[@id="cpn_desc"]/text()')
interest

In [ ]:
cb1 = get_interest_rate('113049')

In [ ]:
cb1

In [ ]:
cb2 = get_interest_rate('120003')

In [ ]:
cb3 = get_interest_rate('132018')

In [ ]:
cb4 = get_interest_rate('128125')

In [ ]:
re.findall(r'(?<!\d|\.)\d+(?:\.\d+)%', cb4[0])

In [ ]:
re.findall(r'(?<!\d|\.)\d+(?:\.\d+)%', cb3[0])

In [ ]:
re.findall(r'(?<!\d|\.)\d+(?:\.\d+)%', cb2[0])

In [ ]:
re.findall(r'(?<!\d|\.)\d+(?:\.\d+)%', cb1[0])

In [ ]:
#data = get_dat()
save_interest_rate(data)

In [ ]:
get_interest_rate('113569')

In [ ]:
detials_url = 'https://www.jisilu.cn/data/convert_bond_detail/'+'127035'
detials_response = requests.get(detials_url)
html = detials_response.content.decode("utf-8")
html = etree.HTML(html)
interest_text = html.xpath('.//td[@id="cpn_desc"]/text()')

In [ ]:
pattern = re.compile(r'(?<!\d|\.)\d+(?:\.\d+)?\s?%')  #get proper format number
interest = pattern.findall(interest_text[0])
interest

In [ ]:
interest2 = [rate.strip("?\s%") for rate in interest]
interest2

In [ ]:
pattern = re.compile(r'(?<!\d|\.)\d+(?:\.\d+)?')  #get proper format number
interest2 = pattern.findall(interest_text[0])
interest2

In [265]:
# Fill in your details here to be posted to the login form.
payload = {
'user_name': 'a39856ed4377d1b92cffb0677febf5b8',
'password': 'ca5572b5ffe8c33746727709bc7d6c6f'
}
headers = {
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36",
#"User-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.106 Safari/537.36 OPR/38.0.2220.41",
"Cookie": "kbzw__Session=l313fggdcjpnd7eineqilm6lf4; kbz_newcookie=1; kbzw_r_uname=ijayden; Hm_lvt_164fe01b1433a19b507595a43bf58262=1627073048,1627079966,1627115858,1627117055; Hm_lpvt_164fe01b1433a19b507595a43bf58262=1627118717"
}
loginUrl="https://www.jisilu.cn/account/ajax/login_process/"
r = requests.post(loginUrl,data=payload)

newUrl ="https://www.jisilu.cn/data/cbnew/cb_list/?___jsl=LST___t=1584777951900"
#最简单的爬虫请求.也可以加上headers字段，防止部分网址的反爬虫机制
response = requests.get(newUrl,cookies=r.cookies)
data = response.content.decode("utf-8")
dat = json.loads(data)

In [268]:
dat['rows'][300]['id']

'113604'

In [271]:
for row in dat['rows']:
    if row['id'] == '113607':
        for key,val in row['cell'].items():
            print(key,val)

bond_id 113607
bond_nm 伟20转债
stock_id sh603568
stock_nm 伟明环保
btype C
convert_price 21.71
convert_price_valid_from 2021-07-06
convert_dt 2021-05-06
maturity_dt 2026-11-01
next_put_dt 2024-11-01
put_dt None
put_notes None
put_price 100.000
put_inc_cpn_fl y
put_convert_price_ratio 56.29
put_count_days 30
put_total_days 30
put_real_days 0
repo_discount_rt 0.00
repo_valid_from None
repo_valid_to None
turnover_rt 3.87
redeem_price 110.000
redeem_inc_cpn_fl n
redeem_price_ratio 130.000
redeem_count_days 20
redeem_total_days 30
redeem_real_days 13
redeem_dt None
redeem_flag X
orig_iss_amt 12.000
curr_iss_amt 11.436
rating_cd AA
issuer_rating_cd AA
guarantor 无担保
ssc_dt None
esc_dt None
sc_notes None
market_cd shmb
force_redeem None
real_force_redeem_price None
convert_cd 113607
repo_cd 113607
ration 0.9540
ration_cd 753568
apply_cd 754568
online_offline_ratio None
qflag N
qflag2 N
ration_rt 91.110
fund_rt buy
margin_flg R
lt_bps 
pb 5.88
pb_flag N
total_shares 1256542551.0
float_shares 12565425

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import scipy.stats as stats
from mpl_toolkits.mplot3d import Axes3D
def func1(x, a, b, c, d):
    r =  a * np.exp(-((x[0] - b) ** 2 + (x[1] - d) ** 2) / (2 * c ** 2))
    return r.ravel()

# 生成原始数据
x1 = np.linspace(0, 10, 10).reshape(1, -1)
x2 = np.linspace(0, 10, 10).reshape(1, -1)
x = np.append(x1, x2, axis=0)
X, Y = np.meshgrid(x1, x2)
XX = np.expand_dims(X, 0)
YY = np.expand_dims(Y, 0)
xx = np.append(XX, YY, axis=0)
y = func1(xx, 10, 5, 2, 5)
# 对原始数据增加噪声
yn = y + 0.002 * np.random.normal(size=xx.shape[1] * xx.shape[2])

# 使用curve_fit函数拟合噪声数据
popt, pcov = curve_fit(func1, xx, yn)

# popt返回最拟合给定的函数模型func的参数值
print(popt)

fig = plt.figure('拟合图')
ax = Axes3D(fig)
X, Y = np.meshgrid(x1, x2)
XX = np.expand_dims(X, 0)
YY = np.expand_dims(Y, 0)
xx = np.append(XX, YY, axis=0)
R = func1(xx, *popt)
# R, _ = np.meshgrid(R, x1)
# y = func1(xx, 10, 5, 2, 5)
# # 对原始数据增加噪声
# yn = y = 0.002 * np.random.normal(size=xx.shape[1] * xx.shape[2])
R = R.reshape(10, 10)
yn = yn.reshape(10, 10)
ax.plot_surface(X, Y, R, rstride=1, cstride=1, cmap='rainbow')
ax.plot_surface(X, Y, yn, rstride=1, cstride=1, color='red')

plt.show()

In [ ]:
x[0]

In [ ]:
x[1]

In [ ]:
x.shape

In [ ]:
yn.shape

In [ ]:
y

In [ ]:
x = np.linspace(0,6)
y1 = x**0.5
y2 = x**0.25
y3 = x**0.1

In [ ]:
plt.plot(x,y1,color='red')
plt.plot(x,y2,color='skyblue')
plt.plot(x,y3,color='green')
plt.show()

In [ ]:
11.9*6**0.25

In [ ]:
5.2*6**0.5

In [ ]:
import tushare as ts

In [ ]:
pro = ts.pro_api('c914778bc260d3911212f2a37620ffa89ca4706e44fd557673b509a2')

In [ ]:
XSG_URL = '%sdatainterface.%s/EM_DataCenter/%s?type=FD&sty=BST&st=3&sr=true&fd=%s&stat=%s'
XSG_COLS = ['id','code', 'name', 'date', 'count', 'ratio']
ZF = '%s%s/tsdata/%s.csv'
FORMAT = lambda x: '%.2f' % x
P_TYPE = {'http': 'http://', 'ftp': 'ftp://'}
DOMAINS = {'sina': 'sina.com.cn', 'sinahq': 'sinajs.cn',
           'ifeng': 'ifeng.com', 'sf': 'finance.sina.com.cn',
            'vsf': 'vip.stock.finance.sina.com.cn',
            'idx': 'www.csindex.com.cn', '163': 'money.163.com',
            'em': 'eastmoney.com', 'sseq': 'query.sse.com.cn',
            'sse': 'www.sse.com.cn', 'szse': 'www.szse.cn',
            'oss': 'file.tushare.org', 'idxip':'115.29.204.48',
            'shibor': 'www.shibor.org', 'mbox':'www.cbooo.cn',
            'tt': 'gtimg.cn', 'gw': 'gw.com.cn',
            'v500': 'value500.com', 'sstar': 'stock.stockstar.co    m',
            'dfcf': 'nufm.dfcfw.com'}

PAGES = {'fd': 'index.phtml', 'dl': 'downxls.php', 'jv': 'json_v2.php',
         'cpt': 'newFLJK.php', 'ids': 'newSinaHy.php', 'lnews':'rollnews_ch_out_interface.php',
         'ntinfo':'vCB_BulletinGather.php', 'hs300b':'000300cons.xls',
         'hs300w':'000300closeweight.xls','sz50b':'000016cons.xls',         
         'dp':'all_fpya.php', '163dp':'fpyg.html',
         'emxsg':'JS.aspx', '163fh':'jjcgph.php',
         'newstock':'vRPD_NewStockIssue.php', 'zz500b':'000905cons.xls',
         'zz500wt':'000905closeweight.xls',
         't_ticks':'vMS_tradedetail.php', 'dw': 'downLoad.html',
         'qmd':'queryMargin.do', 'szsefc':'ShowReport.szse',
         'ssecq':'commonQuery.do', 'sinadd':'cn_bill_download.php', 'ids_sw':'SwHy.php',
         'idx': 'index.php', 'index': 'index.html'}

In [ ]:
year = datetime.today().year
month = datetime.today().month
year = 2022
month = 2

In [ ]:
request = requests.get(XSG_URL%(P_TYPE['http'], DOMAINS['em'],PAGES['emxsg'], year, month))

In [ ]:
def get_restricted_shares(stock_id):
    url = 'http://datacenter-web.eastmoney.com/api/data/v1/get?callback=jQuery112305678710086023377_1630414549986&sortColumns=FREE_DATE&sortTypes=-1&pageSize=500&pageNumber=1&reportName=RPT_LIFT_STAGE&filter=(SECURITY_CODE="'+stock_id+'")&columns=SECURITY_CODE,SECURITY_NAME_ABBR,FREE_DATE,CURRENT_FREE_SHARES,ABLE_FREE_SHARES,LIFT_MARKET_CAP,FREE_RATIO,NEW,B20_ADJCHRATE,A20_ADJCHRATE,FREE_SHARES_TYPE,TOTAL_RATIO,NON_FREE_SHARES,BATCH_HOLDER_NUM&source=WEB&client=WEB'
    my_request = requests.get(url)
    my_lines = my_request.content.decode("utf-8")
    p1 = re.compile(r'[(](.*)[)]',re.S)
    my_json = re.findall(p1,my_lines)
    dat = json.loads(my_json[0])
    return pd.DataFrame.from_dict(dat['result']['data'])

In [ ]:
restricted_df = get_restricted_shares('000001')

In [ ]:
restricted_df

In [ ]:
restricted_df[restricted_df['FREE_SHARES_TYPE'].str.contains("增发")]

In [ ]:
request = requests.get(XSG_URL%(P_TYPE['http'], DOMAINS['em'],PAGES['emxsg'], year, month))

In [ ]:
json.loads(my_lines)

In [ ]:
stock_id = '000538'
r = requests.get('https://vip.stock.finance.sina.com.cn/corp/go.php/vISSUE_AddStock/stockid/'+stock_id+'.phtml')

In [ ]:

temp_df = pd.read_html(r.text)[12]
if temp_df.at[0, 0] == '对不起，暂时没有相关增发记录':
    print(f"股票 {stock} 无增发记录")
big_df = pd.DataFrame()
for i in range(int(len(temp_df.at[0, 1]) / 10)):
    temp_df = pd.read_html(r.text)[13+i].iloc[:, 1]
    big_df[temp_df.name.split(" ")[1].split("：")[1][:10]] = temp_df 
    big_df = big_df.T
    #big_df.columns = ["发行方式", "发行价格", "实际公司募集资金总额", "发行费用总额", "实际发行数量"]

In [ ]:
big_df

In [ ]:
pd.read_html(r.text)[14].iloc[:,1]

In [ ]:
r.text

In [ ]:
XSG_URL%(P_TYPE['http'], DOMAINS['em'],PAGES['emxsg'], year, month)

In [ ]:
lines = request.content.decode("utf-8")
#html = etree.HTML(html)
#interest_text = html.xpath('.//td[@id="cpn_desc"]/text()')

In [ ]:
da = lines[3:len(lines)-3]
list =  []
for row in da.split('","'):
    list.append([data for data in row.split(',')])
df = pd.DataFrame(list)
df = df[[0,1, 3, 4, 5, 6]]
for col in [5, 6]:
    df[col] = df[col].astype(float)
df[5] = df[5]/10000
df[6] = df[6]*100
df[5] = df[5].map(FORMAT)
df[6] = df[6].map(FORMAT)
df.columns = XSG_COLS

In [ ]:
df

In [ ]:
df.loc[df['id'].str.startswith('1')]

In [ ]:
df = pro.share_float(ts_code='300532.SZ')

In [ ]:
df

In [ ]:
df = pro.share_float(ts_code='000157.SZ')

In [ ]:
df

In [ ]:
df['float_ratio'].sum()

In [ ]:
pro.share_float(ts_code='600562.SH')

In [301]:
data1 = np.zeros([10,5,4])

In [302]:
alphabet = np.array(['A', 'T', 'C', 'G'])
sequence = ['A', 'T', 'C', 'G','T']
seq = np.array(sequence, dtype = '|U1').reshape(-1,1)
seq_data = (seq == alphabet).astype(np.float32)

In [303]:
seq_data

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.]], dtype=float32)

In [304]:
data1[0,:,:] = seq_data

In [305]:
sequence = ['T', 'C', 'G', 'A','T']
seq = np.array(sequence, dtype = '|U1').reshape(-1,1)
seq_data = (seq == alphabet).astype(np.float32)

In [338]:
complement = {'A':'T','T':'A','C':'G','G':'C'}

In [339]:
newseq = [complement[base] for base in sequence]
newseq.reverse()

In [340]:
newseq

['A', 'T', 'C', 'G', 'A']

In [306]:
data1[1,:,:] = seq_data

In [307]:
data1

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.]],

       [[0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]

In [309]:
bl = [True,True,False,False,False,False,False,False,False,False]
bl2 = [1,1,0,0,0,0,0,0,0,0]
data1[bl]

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.]],

       [[0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.]]])

In [310]:
data1[bl2]

array([[[0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.]],

       [[0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.]],

       [[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.]],

       [[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.]],

       [[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.]],

       [[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.]],

       [[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.]],

       [[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]

In [313]:
bl3 = np.zeros(10,dtype=bool)

In [314]:
bl3[:2] = True

In [320]:
data1 = data1[bl3]

In [321]:
data2

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.]],

       [[0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.]]])

In [322]:
data1

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.]],

       [[0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.]]])

In [323]:
data1.shape

(2, 5, 4)

In [326]:
np.empty(10,dtype='object')

array([None, None, None, None, None, None, None, None, None, None],
      dtype=object)

In [336]:
bl3.reshape([-1,1])

array([[ True],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False]])

In [333]:
bl3.shape

(10,)

In [337]:
bl3

array([ True,  True, False, False, False, False, False, False, False,
       False])

In [344]:
bl3[::-1]

array([False, False, False, False, False, False, False, False,  True,
        True])

In [346]:
x = np.array(newseq)

In [347]:
x

array(['A', 'T', 'C', 'G', 'A'], dtype='<U1')

In [353]:
x = x[::-1]

In [354]:
x

array(['A', 'G', 'C', 'T', 'A'], dtype='<U1')